In [1]:
from process import process
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='darkgrid')

/Users/miron/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_train = pd.read_csv('../../data/credit_card_fraud_detection/fraudTrain.csv', index_col=0)
df_test = pd.read_csv('../../data/credit_card_fraud_detection/fraudTest.csv', index_col=0)
print(f'Rows: {df_train.shape[0]} | Columns: {df_train.shape[1]} (Train)')
print(f'Rows: {df_test.shape[0]} | Columns: {df_test.shape[1]} (Test)')

Rows: 1296675 | Columns: 22 (Train)
Rows: 555719 | Columns: 22 (Test)


In [3]:
df = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

In [4]:
df_processed = process(df)
df_processed.head()

,category,amt,gender,city,state,lat,long,city_pop,job,merch_lat,...,hour,num_of_trans,num_of_unique_merchant,lat_std,long_std,country,mean_amt_per_category,distance_km,age,amt_above_mean
0,0.013039,4.97,1,0.003758,0.004521,-366.119680,540.761328,3495,0.003320,-373.634559,...,0,2927,660,0.0,0.0,United States,80.181370,75.267255,24.0,NaN
1,0.012645,107.23,1,0.002160,0.004660,1059.778200,-2646.266572,149,0.002472,1089.973416,...,0,4362,681,0.0,0.0,United States,116.640146,30.265999,34.0,NaN
2,0.002177,220.11,0,0.010884,0.004107,313.154960,-2134.335520,4154,0.021534,421.124673,...,0,735,431,0.0,0.0,United States,64.142968,108.365491,50.0,NaN
3,0.004106,45.00,0,0.020188,0.004106,763.978696,-2121.581350,1939,0.005461,853.450031,...,0,743,423,0.0,0.0,United States,63.477271,97.400118,45.0,NaN
4,0.002819,41.96,0,0.004449,0.006538,-105.419372,688.372346,99,0.004449,-77.110807,...,0,2922,652,0.0,0.0,United States,62.676479,76.870522,26.0,NaN


In [5]:
X = df_processed.drop('is_fraud', axis=1)
y = df_processed['is_fraud']

In [ ]:
!conda install -c conda-forge imbalanced-learn

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::jupyterlab_server==2.22.0=py311hca03da5_0
  - defaults/osx-arm64::bokeh==3.2.1=py311hb6e6a13_0
  - defaults/osx-arm64::_anaconda_depends==2023.09=py311_openblas_1
  - defaults/noarch::argon2-cffi==21.3.0=pyhd3eb1b0_0
  - defaults/osx-arm64::anaconda-catalogs==0.2.0=py311hca03da5_0
  - defaults/osx-arm64::notebook==6.5.4=py311hca03da5_1
  - defaults/osx-arm64::twisted==22.10.0=py311h80987f9_0
  - defaults/osx-arm64::jupyter_server_ydoc==0.8.0=py311hca03da5_1
  - defaults/osx-arm64::dask==2023.6.0=py311hca03da5_0
  - defaults/osx-arm64::s3fs==2023.4.0=py311hca03da5_0
  - defaults/osx-arm64::datasets==2.12.0=py311hca03da5_0
  - defaults/osx-arm64::huggingface_hub==0.15.1=py311hca03da5_0
  - defaults/osx-arm64::anaconda-navigator==2.5.0=py311hca03da5_0
  - conda-forge/noarch::jupyterlab==3.6.7=pyhd8ed1ab_0
  - defaul

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 5, 10]
}

model = XGBClassifier()

grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=3, n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)

best_estimator = grid_search.best_estimator_

y_pred = best_estimator.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

y_scores = best_estimator.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

auc_pr = auc(recall, precision)
print("AUC-PR:", auc_pr)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label='Precision-Recall curve (AUC = %0.2f)' % auc_pr)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()

In [ ]:
import pickle

model_file_path = 'xgboost_model_fraud_detection.pkl'

with open(model_file_path, 'wb') as f:
    pickle.dump(best_estimator, f)

print("Model is saved to", model_file_path)